In [3]:
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy import select, bindparam
engine = create_engine("sqlite+pysqlite:///MyFirstDB", echo=True, future=True)

In [ ]:
# "commit as you go"
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x,y) VALUES (:x,:y)"),
        [{"x":1,"y":1},{"x":2,"y":4}]
    )
    conn.commit()

In [ ]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT  INTO some_table (x,y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}] 
    )

In [5]:
#Fetching rows
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM some_table"))
    for row in result:
        print(f"x:{row.x} y:{row.y}")

2021-07-28 12:39:32,881 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-28 12:39:32,882 INFO sqlalchemy.engine.Engine SELECT * FROM some_table
2021-07-28 12:39:32,883 INFO sqlalchemy.engine.Engine [generated in 0.00167s] ()
x:1 y:1
x:2 y:4
x:6 y:8
x:9 y:11
x:11 y:12
x:13 y:15
2021-07-28 12:39:32,886 INFO sqlalchemy.engine.Engine ROLLBACK


In [ ]:
#Sending Parameters
with engine.connect() as conn:
    result = conn.execute(
        text("SELECT x,y FROM some_table WHERE y > :y"),
        {"y":2}
    )
    for row in result:
        print(f"x:{row.x} y:{row.y}")

In [ ]:
#Sending Multiple Parameters
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x,y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}]
    )
    conn.commit()

In [ ]:
#Bundling Parameters with a Statement
from sqlalchemy.orm import Session
#-----statement--#
stmt = text("SELECT x,y FROM some_table WHERE y>:y ORDER BY x,y").bindparams(y=6)
#-----------------#
with Session(engine) as session:
    result = session.execute(stmt)
    for row in result:
        print(f"x:{row.x} y:{row.y}")

In [ ]:
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y":11}, {"x": 13, "y": 15}]
    )
    session.commit

In [7]:
#Setting up MetaData with Table objects
from sqlalchemy import MetaData

metadata = MetaData()

In [8]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String(30)),
    Column('fullname',String)
)

In [9]:
#Declaring simple Constraints
from sqlalchemy import ForeignKey
address_table = Table (
    "address",
    metadata,
    Column('id', Integer, primary_key=True),
    Column('user_id', ForeignKey('user_account.id'), nullable= False),
    Column('email_address', String, nullable= False)
)

In [ ]:
metadata.create_all(engine)

In [ ]:
some_table = Table("some_table", metadata, autoload_with= engine)

In [ ]:
some_table

## Working with Data

In [ ]:
from sqlalchemy import insert
stmt = insert(user_table).values(name= 'spongebob', fullname= "Spongebob Squarepants")
print(stmt)


In [ ]:
compiled = stmt.compile()
compiled.params

In [ ]:
with engine.connect() as conn:
    result= conn.execute(stmt)
    conn.commit()

In [ ]:
with engine.connect() as conn:
    result= conn.execute(
        insert(user_table),
        [
            {"name":"sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"}
        ]
    )
    conn.commit()

In [ ]:
from sqlalchemy import select, bindparam
scalar_subquery = (
    select(user_table.c.id).
    where(user_table.c.name==bindparam('username')).
    scalar_subquery()
)



In [ ]:
with engine.connect() as conn:
   result = conn.execute(
       insert(address_table).values(user_id=scalar_subquery),
       [
           {"username": 'spongebob', "email_address": "spongebob@sqlalchemy.org"},
           {"username": 'sandy', "email_address": "sandy@sqlalchemy.org"},
           {"username": 'sandy', "email_address": "sandy@squirrelpower.org"},
       ]
   )
   conn.commit()

In [ ]:
select_stmt = select(user_table.c.id,user_table.c.name + "@aol.co")
insert_stmt = insert(address_table).from_select(
    ["user_id","email_address"], select_stmt)
print(insert_stmt)

In [ ]:
insert_stmt= insert(address_table).returning(
    address_table.c.id, address_table.c.email_address)
print(insert_stmt)

## Selecting Rows with Core or ORM

In [10]:
stmt = select(user_table).where(user_table.c.name == 'spongebob')
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [11]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2021-07-28 13:34:41,353 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-28 13:34:41,355 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2021-07-28 13:34:41,356 INFO sqlalchemy.engine.Engine [generated in 0.00388s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
2021-07-28 13:34:41,360 INFO sqlalchemy.engine.Engine ROLLBACK


In [ ]:
print(select(user_table.c.name, user_table.c.fullname))

In [ ]:
from sqlalchemy import literal_column
stmt=(
    select(
        ("Username:" + user_table.c.name).label("username"),
    ).order_by(user_table.c.name)
)
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.username}")